# 3장 컨테이너 설정

## 3.1 자원 제한
### 3.1.1. CPU 우선 순위
- ```-c``` 플래그로 설정
- 모든 컨테이너는 동일한 우선순위가 지정됨
- 임의의 우선순위를 지정

```
docker run -c 10 -it ubuntu /bin/bash

```

## 3.1.2 메모리 제한

- ```-m``` 플래그로 설정
- ```-m <사이즈><단위>```

```
docker rum -m 1000m -it ubuntu /bin/bash
```

- cpu와 memory는 다음 위치에서 확인 및 변경 가능

```
cat /sys/fs/cgroup/memory/<container_id>/memory.limit_in_bytes
cat /sys/fs/cgroup/memory/<container_id>//cpu.shares
```

- 런타임 변경 가능

```
echo <크기> > /sys/fs/cgroup/memory/<container_id>/memory.limit_in_bytes
echo <우선순위> > cat /sys/fs/cgroup/memory/<container_id>/cpu.shares
```

- cgroup의 위치 확인

```
grep -w cgroup /proc/mounts | grep -w memory

```

### 3.1.3 디스크 제한
- device-mapper를 사용
- 운영체제의 마운트 단계에서 설정

## 3.2 컨테이너 내부 데이터 관리

- ```-v``` 옵션으로 구성 가능
- 하나의 볼륨은 여러 컨테이너가 공유할 수 있음
- 참조하는 컨테이너가 없으면 소멸

### 3.2.1 데이터 전용 컨테이넠

- ```--volumes-froms```을 사용하여 다른 컨테이너가 생성한 볼륨을 참조할 수 있음

```
docker run -v /data/db --name data-only mongo echo "MongoDB stores all data in /data/db"
docker run --volumes-from data-only -p 27017:27017 -name mongodb-server mongo mongod

```

## docker link 정리

- link옵션의 형식
  - ```--link <container_name>:<alias_name>```
- link 옵션을 설정하여 docker 컨테이너를 생성하면 link에 등록된 컨테이너 명, 해당 컨테이너 id, 컨테이너 alias를 hosts파일에 등록함
- docker가 하나의 host에 운영될 때 ip가 아닌 container 명 혹은 alias로 다른 서버를 참조하기에 유용함

### 데모
- 시나리오
  - 2개의 container를 올림 
    - server
    - client
  
```
taewan@jupyter_docbase $docker run -dit --name con1 ubuntu /bin/bash 
5a7e05adcdf8f16125927b67f6583764648d39de3f3352460ca6118958aef976
taewan@jupyter_docbase $docker run -dit --name con2 --link con1:con2 ubuntu /bin/bash
632a8a07b4794a5b3d77403dbbb8e32a4decb6e2631bff1dadb157d524a7370c
```

- con2에 접속 후 hosts파일 확인
```
taewan@jupyter_docbase $docker attach con2
root@632a8a07b479:/# cat /etc/hosts 
127.0.0.1	localhost
::1	localhost ip6-localhost ip6-loopback
fe00::0	ip6-localnet
ff00::0	ip6-mcastprefix
ff02::1	ip6-allnodes
ff02::2	ip6-allrouters
172.17.0.3	con2 5a7e05adcdf8 con1
172.17.0.4	632a8a07b479
root@632a8a07b479:/#
```